<a href="https://colab.research.google.com/github/qiwang2629/-latex/blob/main/sdxl_v1.0_controlnet_comfyui_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

# ✅ 安装 PyTorch（兼容 Triton）
!pip install torch==2.1.0 torchvision==0.16.0 torchaudio==2.1.0 --index-url https://download.pytorch.org/whl/cu118

# ✅ 补上 CLIPVision 所需 transformers
!pip install transformers==4.31.0

# ✅ 安装系统库 & tcmalloc
!apt -y update -qq
!wget https://github.com/camenduru/gperftools/releases/download/v1.0/libtcmalloc_minimal.so.4 -O /content/libtcmalloc_minimal.so.4
%env LD_PRELOAD=/content/libtcmalloc_minimal.so.4
!apt -y install -qq aria2

# ✅ 安装 Python 依赖
!pip install -q xformers==0.0.20 triton==2.0.0
!pip install -q addict yapf fvcore omegaconf mediapipe==0.10.21

# ✅ 克隆 ComfyUI 与 ControlNet 插件
!git clone -b v1.5 https://github.com/camenduru/ComfyUI
%cd /content/ComfyUI
!pip install -q -r requirements.txt

!git clone -b v1.5 https://github.com/camenduru/comfy_controlnet_preprocessors /content/ComfyUI/custom_nodes/comfy_controlnet_preprocessors
%cd /content/ComfyUI/custom_nodes/comfy_controlnet_preprocessors
!python install.py --no_download_ckpts
%cd /content/ComfyUI

# ✅ 启动 Cloudflare 隧道，替代 --share
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O /content/cloudflared-linux-amd64 && chmod 777 /content/cloudflared-linux-amd64
import atexit, requests, subprocess, time, re
from random import randint
from threading import Timer
from queue import Queue

def cloudflared(port, metrics_port, output_queue):
    atexit.register(lambda p: p.terminate(), subprocess.Popen(
        ['/content/cloudflared-linux-amd64', 'tunnel', '--url', f'http://127.0.0.1:{port}', '--metrics', f'127.0.0.1:{metrics_port}'],
        stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT))
    attempts, tunnel_url = 0, None
    while attempts < 10 and not tunnel_url:
        attempts += 1
        time.sleep(3)
        try:
            tunnel_url = re.search("(?P<url>https?:\/\/[^\s]+.trycloudflare.com)",
                                   requests.get(f'http://127.0.0.1:{metrics_port}/metrics').text).group("url")
        except:
            pass
    if not tunnel_url:
        raise Exception("❌ 无法连接 Cloudflare")
    output_queue.put(tunnel_url)

output_queue, metrics_port = Queue(), randint(8100, 9000)
thread = Timer(2, cloudflared, args=(8188, metrics_port, output_queue))
thread.start()
thread.join()
tunnel_url = output_queue.get()
os.environ['webui_url'] = tunnel_url
print(f"✅ ComfyUI 启动成功，请点击访问：{tunnel_url}")

# ✅ 下载模型
!mkdir -p /content/ComfyUI/models/checkpoints
!mkdir -p /content/ComfyUI/models/controlnet
!mkdir -p /content/ComfyUI/models/upscale_models
!mkdir -p /content/ComfyUI/models/vae

# 🔹 SDXL Base
!aria2c -q -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/sd_xl_base_1.0/resolve/main/sd_xl_base_1.0_0.9vae.safetensors -d /content/ComfyUI/models/checkpoints -o sd_xl_base_1.0.safetensors

# 🔹 ControlNet OpenPose
!aria2c -q -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/controlnet-sdxl-1.0/resolve/main/OpenPoseXL2.safetensors -d /content/ComfyUI/models/controlnet -o OpenPoseXL2.safetensors

# 🔹 Real-ESRGAN Upscaler
!aria2c -q -c -x 16 -s 16 -k 1M https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.1/RealESRGAN_x2plus.pth -d /content/ComfyUI/models/upscale_models -o RealESRGAN_x2plus.pth

# 🔹 VAE
!aria2c -q -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/sdxl_vae/resolve/main/sdxl_vae.safetensors -d /content/ComfyUI/models/vae -o sdxl_vae.vae.safetensors

# ✅ 启动 ComfyUI 主程序
!python main.py --dont-print-server
